In [1]:
import os

import pandas as pd
import numpy as np
import torch
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

##### ----------

In [2]:
pip install catboost xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder


from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.utils.class_weight import compute_sample_weight

import numpy as np
SEED = 42
n_splits = 5
train = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/train.csv')
test = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/test.csv')
sample = pd.read_csv('/kaggle/input/child-mind-institute-problematic-internet-use/sample_submission.csv')

def preprocess_train_data(data):

    # Drop 'id' and separate numeric and categorical data

    data_no_id = data.drop(columns=['id'])
    data_no_id = data_no_id[data_no_id['Physical-Systolic_BP'] > data_no_id['Physical-Diastolic_BP']].reset_index(drop=True)
    data_no_id = data_no_id[(data_no_id['Physical-Systolic_BP'] - data_no_id['Physical-Diastolic_BP'])> 25].reset_index(drop=True)
    numeric_data = data_no_id.select_dtypes(include=['number']).copy()

    categorical_data = data_no_id.select_dtypes(exclude=['number']).copy()

    

    # Drop PCIAT-PCIAT_Total and sii before applying KNN

    numeric_data = numeric_data.drop(columns=['PCIAT-PCIAT_Total', 'sii'], errors='ignore')

    

    # Impute categorical data with mode

    for col in categorical_data.columns:

        categorical_data[col].fillna(categorical_data[col].mode()[0], inplace=True)

    

    # Encode categorical data

    encoder = OrdinalEncoder()

    categorical_encoded = encoder.fit_transform(categorical_data)

    categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=categorical_data.columns)

    

    # Apply KNN Imputer to numeric data

    knn_imputer = KNNImputer(n_neighbors=5)

    numeric_imputed = pd.DataFrame(knn_imputer.fit_transform(numeric_data), columns=numeric_data.columns)

    

    # Round PCIAT-PCIAT_[num] columns and calculate PCIAT-PCIAT_Total

    pciat_columns = [col for col in numeric_imputed.columns if col.startswith('PCIAT-PCIAT_')]

    for col in pciat_columns:

        numeric_imputed[col] = numeric_imputed[col].round().astype(int)

    numeric_imputed['PCIAT-PCIAT_Total'] = numeric_imputed[pciat_columns].sum(axis=1)

    

    # Determine sii based on PCIAT-PCIAT_Total

    def calculate_sii(total):

        if total <= 30:

            return 0  # None

        elif 31 <= total <= 49:

            return 1  # Mild

        elif 50 <= total <= 79:

            return 2  # Moderate

        else:

            return 3  # Severe



    numeric_imputed['sii'] = numeric_imputed['PCIAT-PCIAT_Total'].apply(calculate_sii)



    # Combine numeric and categorical data

    processed_data = pd.concat([data['id'].reset_index(drop=True), numeric_imputed, categorical_encoded_df], axis=1)

    

    return processed_data



# Function to preprocess test data

def preprocess_test_data(data):

    # Drop 'id' and separate numeric and categorical data

    data_no_id = data.drop(columns=['id'])

    numeric_data = data_no_id.select_dtypes(include=['number']).copy()

    categorical_data = data_no_id.select_dtypes(exclude=['number']).copy()

    

    # Impute categorical data with mode

    for col in categorical_data.columns:

        categorical_data[col].fillna(categorical_data[col].mode()[0], inplace=True)

    

    # Encode categorical data

    encoder = OrdinalEncoder()

    categorical_encoded = encoder.fit_transform(categorical_data)

    categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=categorical_data.columns)

    

    # Apply KNN Imputer to numeric data

    knn_imputer = KNNImputer(n_neighbors=5)

    numeric_imputed = pd.DataFrame(knn_imputer.fit_transform(numeric_data), columns=numeric_data.columns)



    # Combine numeric and categorical data

    processed_data = pd.concat([data['id'].reset_index(drop=True), numeric_imputed, categorical_encoded_df], axis=1)

    

    return processed_data
def feature_engineering(df, training):
    season_cols = [col for col in df.columns if 'Season' in col]
    df = df.drop(season_cols, axis=1) 
    df['BMI_Average'] = df[['Physical-BMI', 'BIA-BIA_BMI']].mean(axis=1)
    df['Fitness_Endurance_Time_Total_Sec'] = df['Fitness_Endurance-Time_Mins'] * 60 + df['Fitness_Endurance-Time_Sec']
    df.drop(['Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec'], axis=1, inplace=True)
    df.drop(['Physical-BMI', 'BIA-BIA_BMI'], axis=1, inplace=True)
    if training:
        pciat_questions = [f'PCIAT-PCIAT_{i:02d}' for i in range(1, 21)]
        df.drop(pciat_questions, axis=1, inplace=True)
        df.drop('PCIAT-PCIAT_Total', axis=1, inplace=True)
    
    
    return df
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df



In [4]:
train = preprocess_train_data(train)
train = feature_engineering(train, True)
test = preprocess_test_data(test)
test = feature_engineering(test, False)

train_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet")
test_ts = load_time_series("/kaggle/input/child-mind-institute-problematic-internet-use/series_test.parquet")

time_series_cols = train_ts.columns.tolist()
time_series_cols.remove("id")

train = pd.merge(train, train_ts, how="left", on='id')
test = pd.merge(test, test_ts, how="left", on='id')

train = train.drop(columns=['id'], axis=1)
test = test.drop(columns=['id'], axis=1)   

featuresCols = ['Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
                'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 
                'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
                'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
                'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
                'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
                'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
                'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
                'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
                'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
                'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 
                'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
                'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
                'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
                'BIA-BIA_TBW', 'PAQ_A-Season', 'PAQ_A-PAQ_A_Total', 'PAQ_C-Season',
                'PAQ_C-PAQ_C_Total', 'SDS-Season', 'SDS-SDS_Total_Raw',
                'SDS-SDS_Total_T', 'PreInt_EduHx-Season',
                'PreInt_EduHx-computerinternet_hoursday', 'sii']

featuresCols += time_series_cols

train = train.dropna(subset='sii')
print(train.shape)
print(test.shape)


100%|██████████| 2/2 [00:00<00:00, 11.33it/s]

(2848, 143)
(20, 142)


In [5]:

def create_mapping(column, dataset):
    unique_values = dataset[column].unique()
    return {value: idx for idx, value in enumerate(unique_values)}

def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

def TrainML(model_class, test_data):
    X = train.drop(['sii'], axis=1)
    y = train['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission



In [6]:

# Model parameters for LightGBM
LGBM_Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  
    'lambda_l2': 0.01  
}


# XGBoost parameters
XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  
    'reg_lambda': 5,  
    'random_state': SEED
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10  # Increase this value
}

# Create model instances
Light = LGBMRegressor(**LGBM_Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model)
])

# Train the ensemble model
Submission2 = TrainML(voting_model, test)

Submission2.to_csv('submission.csv', index=False)

Training Folds: 100%|██████████| 5/5 [00:37<00:00,  7.51s/it]

Mean Train QWK --> 0.7767
Mean Validation QWK ---> 0.4256


----> || Optimized QWK SCORE ::  0.489
